In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import warnings
import matplotlib.cbook
import matplotlib as mpl
import matplotlib.image as mpimg
import time
import itertools
import seaborn as sns
import os
import json
from sympy import *
from scipy.integrate import odeint
from scipy.interpolate import interp1d
from scipy.fftpack import rfft,fft,fftfreq,rfftfreq
from scipy.signal import medfilt, find_peaks
from scipy.io import loadmat
from scipy import stats
from statsmodels.stats.multitest import fdrcorrection
from matplotlib.lines import Line2D
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit,brute,basinhopping
import matplotlib.gridspec as gridspec
from sklearn.decomposition import PCA, NMF
from sklearn.cluster import KMeans
import math
from itertools import combinations
import matplotlib.cm as cm
from sklearn.manifold import TSNE
from string import ascii_uppercase as abc; abc = abc
try:
    from tqdm.notebook import tqdm
except:
    from tqdm import tqdm_notebook as tqdm
from matplotlib.axes._axes import _log as matplotlib_axes_logger
from itertools import compress, product
import random
from scipy.interpolate import InterpolatedUnivariateSpline as spline_interp
import warnings
warnings.filterwarnings("ignore")
import matplotlib.patheffects as PathEffects
matplotlib_axes_logger.setLevel('ERROR')
np.seterr(divide='ignore', invalid='ignore')

def_color  = [x['color'] for x in plt.rcParams['axes.prop_cycle']]

init_printing()
plt.rc('font', size=8) 
plt.rc('xtick', labelsize=8) 
plt.rc('ytick', labelsize=8) 
plt.rc('axes', titlesize=8) 
matplotlib.rcParams["figure.dpi"] = 100
        
def to_dict(keys,values):
    return dict(zip(keys,values))


In [12]:
x1,x2,x3,m1,m2,a1,a2,a3,b1,b2,b3,c1,c2,d1,d2 = symbols('x_1,x_2,x_3,m_1,m_2,a_1,a_2,a_3,b_1,b_2,b_3,c_1,c_2,d_1,d_2',positive=True)
t,a_p,a_t,p= symbols('t,a_P,a_T,p' ,positive=True)

X1       = Function('x_1',positive=True)(t)
X2       = Function('x_2',positive=True)(t)
X3       = Function('x_3',positive=True)(t)
M1       = Function('M_1',positive=True)(t)
M2       = Function('M_2',positive=True)(t)

dx1      = b1/X3       - a1*X1  
dx2      = b2*M1*X1/X3 - a2*X2       
dx3      = b3*M2*X2    - a3*X3    
dm1      = M1*(c1/X3   - d1)
dm2      = M2*(c2*X2   - d2)

deqs     = Matrix([Eq(X1.diff(t), dx1),
                   Eq(X2.diff(t), dx2),
                   Eq(X3.diff(t), dx3),
                   Eq(M1.diff(t), dm1),
                   Eq(M2.diff(t), dm2)])

display(deqs)

⎡    d                         b₁      ⎤
⎢    ──(x₁(t)) = -a₁⋅x₁(t) + ─────     ⎥
⎢    dt                      x₃(t)     ⎥
⎢                                      ⎥
⎢d                       b₂⋅M₁(t)⋅x₁(t)⎥
⎢──(x₂(t)) = -a₂⋅x₂(t) + ──────────────⎥
⎢dt                          x₃(t)     ⎥
⎢                                      ⎥
⎢d                                     ⎥
⎢──(x₃(t)) = -a₃⋅x₃(t) + b₃⋅M₂(t)⋅x₂(t)⎥
⎢dt                                    ⎥
⎢                                      ⎥
⎢    d           ⎛  c₁      ⎞          ⎥
⎢    ──(M₁(t)) = ⎜───── - d₁⎟⋅M₁(t)    ⎥
⎢    dt          ⎝x₃(t)     ⎠          ⎥
⎢                                      ⎥
⎢  d                                   ⎥
⎢  ──(M₂(t)) = (c₂⋅x₂(t) - d₂)⋅M₂(t)   ⎥
⎣  dt                                  ⎦

In [13]:
simp = {a1:b1,a2:b2,a3:b3,c1:d1,c2:d2,d1:a_p,d2:a_t}
stst = to_dict([X1,X2,X3,M1,M2],solve(Matrix([dx1,dx2,dx3,dm1,dm2]).subs(simp),[X1,X2,X3,M1,M2])[0])
display(stst)

In [14]:
lin_deqs = Matrix([Eq(X1.diff(t),dx1.diff(X1).subs(stst)*(X1-1) + dx1.diff(X2).subs(stst)*(X2-1) + dx1.diff(X3).subs(stst)*(X3-1) + dx1.diff(M1).subs(stst)*(M1-1) + dx1.diff(M2).subs(stst)*(M2-1)),
                   Eq(X2.diff(t),dx2.diff(X1).subs(stst)*(X1-1) + dx2.diff(X2).subs(stst)*(X2-1) + dx2.diff(X3).subs(stst)*(X3-1) + dx2.diff(M1).subs(stst)*(M1-1) + dx2.diff(M2).subs(stst)*(M2-1)),
                   Eq(X3.diff(t),dx3.diff(X1).subs(stst)*(X1-1) + dx3.diff(X2).subs(stst)*(X2-1) + dx3.diff(X3).subs(stst)*(X3-1) + dx3.diff(M1).subs(stst)*(M1-1) + dx3.diff(M2).subs(stst)*(M2-1)),
                   Eq(M1.diff(t),dm1.diff(X1).subs(stst)*(X1-1) + dm1.diff(X2).subs(stst)*(X2-1) + dm1.diff(X3).subs(stst)*(X3-1) + dm1.diff(M1).subs(stst)*(M1-1) + dm1.diff(M2).subs(stst)*(M2-1)),
                   Eq(M2.diff(t),dm2.diff(X1).subs(stst)*(X1-1) + dm2.diff(X2).subs(stst)*(X2-1) + dm2.diff(X3).subs(stst)*(X3-1) + dm2.diff(M1).subs(stst)*(M1-1) + dm2.diff(M2).subs(stst)*(M2-1))]).subs(simp)
                
display(lin_deqs)

⎡                d                                                            
⎢                ──(x₁(t)) = -b₁⋅(x₁(t) - 1) - b₁⋅(x₃(t) - 1)                 
⎢                dt                                                           
⎢                                                                             
⎢d                                                                            
⎢──(x₂(t)) = b₂⋅(M₁(t) - 1) + b₂⋅(x₁(t) - 1) - b₂⋅(x₂(t) - 1) - b₂⋅(x₃(t) - 1)
⎢dt                                                                           
⎢                                                                             
⎢        d                                                                    
⎢        ──(x₃(t)) = b₃⋅(M₂(t) - 1) + b₃⋅(x₂(t) - 1) - b₃⋅(x₃(t) - 1)         
⎢        dt                                                                   
⎢                                                                             
⎢                        d                          

In [15]:
Y1       = Function('y_1')(t)
Y2       = Function('y_2')(t)
Y3       = Function('y_3')(t)
Z1       = Function('Z_1')(t)
Z2       = Function('Z_2')(t)

lin_subs = {X1:Y1+1,X2:Y2+1,X3:Y3+1,M1:Z1+1,M2:Z2+1}
lin_deqs = lin_deqs.subs(lin_subs)
display(lin_deqs)

⎡          d                                              ⎤
⎢          ──(y₁(t) + 1) = -b₁⋅y₁(t) - b₁⋅y₃(t)           ⎥
⎢          dt                                             ⎥
⎢                                                         ⎥
⎢d                                                        ⎥
⎢──(y₂(t) + 1) = b₂⋅Z₁(t) + b₂⋅y₁(t) - b₂⋅y₂(t) - b₂⋅y₃(t)⎥
⎢dt                                                       ⎥
⎢                                                         ⎥
⎢     d                                                   ⎥
⎢     ──(y₃(t) + 1) = b₃⋅Z₂(t) + b₃⋅y₂(t) - b₃⋅y₃(t)      ⎥
⎢     dt                                                  ⎥
⎢                                                         ⎥
⎢               d                                         ⎥
⎢               ──(Z₁(t) + 1) = -a_P⋅y₃(t)                ⎥
⎢               dt                                        ⎥
⎢                                                         ⎥
⎢                d                      

In [16]:
qst  = solve([lin_deqs[0].args[1],lin_deqs[1].args[1],lin_deqs[2].args[1]],[Y1,Y2,Y3])
display(qst)

In [17]:
zdeqs = Matrix([Eq(Z1.diff(t),lin_deqs[3].args[1].subs(qst)),
               Eq(Z2.diff(t),lin_deqs[4].args[1].subs(qst))])
display(zdeqs)

⎡d                ⎛Z₁(t)   Z₂(t)⎞ ⎤
⎢──(Z₁(t)) = -a_P⋅⎜───── + ─────⎟ ⎥
⎢dt               ⎝  3       3  ⎠ ⎥
⎢                                 ⎥
⎢d               ⎛Z₁(t)   2⋅Z₂(t)⎞⎥
⎢──(Z₂(t)) = a_T⋅⎜───── - ───────⎟⎥
⎣dt              ⎝  3        3   ⎠⎦

In [18]:
M = Matrix([lin_deqs[3].args[1].subs(qst),lin_deqs[4].args[1].subs(qst)])
J = M.jacobian([Z1,Z2])
display(J)

⎡-a_P    -a_P  ⎤
⎢─────   ───── ⎥
⎢  3       3   ⎥
⎢              ⎥
⎢ a_T   -2⋅a_T ⎥
⎢ ───   ───────⎥
⎣  3       3   ⎦

In [19]:
Z10,Z20   = symbols('Z_{1_0},Z_{2_0}')
ICS       = {Z1.subs(t,0):Z10,Z2.subs(t,0):Z20}
lin_zsols = Matrix(dsolve(zdeqs,ics=ICS))
zsubs     = {Z1:lin_zsols[0].args[1],Z2:lin_zsols[1].args[1]}
lin_sols  = Matrix([Eq(Y1,qst[Y1].subs(zsubs).trigsimp()),
                   Eq(Y2,qst[Y2].subs(zsubs).trigsimp()),
                   Eq(Y3,qst[Y3].subs(zsubs).trigsimp()),
                   Eq(Z1,lin_zsols[0].args[1].trigsimp()),
                   Eq(Z2,lin_zsols[1].args[1].trigsimp())])

In [ ]:
A = symbols('A')
simp_subs = {sqrt(a_p**2 - 8*a_p*a_t+ 4*a_t**2):A}
display(Eq(A,sqrt(a_p**2 - 8*a_p*a_t+ 4*a_t**2)))
display(lin_sols.subs(simp_subs))